# Online Solvers

In [1]:
import os
import pickle
import pandas as pd
from copy import deepcopy

In [2]:
os.chdir("..")

In [3]:
import config
from core import osrm

## Load Data (Payload)

Example data is loaded for a single incoming request to be routed in an existing manifest 

In [4]:
with open("./data/online_payload.pkl", 'rb') as f:
    payload = pickle.load(f)

In [5]:
payload.keys()

dict_keys(['pickup', 'dropoff', 'depot', 'date', 'time_matrix', 'nodes', 'driver_runs', 'booking_id', 'distance_matrix', 'manifests', 'requests'])

In [6]:
len(payload['requests'])

169

In [12]:
def get_pt(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon, action): 
    if action == 'pickup':
        r = {'lat': pickup_lat, 'lon': pickup_lon}
    elif action == 'dropoff':
        r = {'lat': dropoff_lat, 'lon': dropoff_lon}
    else:
        r = {'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON}
    return r
def manifest_report_h(df, payload):
    # for insertion heurisistic
    # df must have columns 'booking_id', 'scheduled_arrival_dt', 'action'
    # metrics: 'vehicle_miles_travelled', 'vehicle_miles_travelled_per_passenger_served', 'number_of_passengers_served',
    # 'vehicle_deadhead_miles'
    # 'percentage_of_deadhead_travel (total distance without passengers / total_distance), 'total_detour_time', 
    
    # first sort by 'scheduled_arrival_dt' so we don't have to assume df is in the correct order initially
    df = df.sort_values(by='scheduled_arrival_dt')
    df['node_id'] = list(range(len(df)))
    
    # calculate 'occupancy' for manifest
    occupancy = [0]
    for i in range(1, len(df)):
        if df.iloc[i]['action'] == 'pickup':
            temp = occupancy[-1] + 1
        elif df.iloc[i]['action'] == 'dropoff':
            temp = occupancy[-1] - 1
        else:
            temp = 0
        occupancy.append(temp)
    df['occupancy'] = occupancy
    
    # need to get geolocation of each point so we can calculate distances, this requires merging with requests    
    df_requests = pd.DataFrame(payload['requests'])
    df = pd.merge(df, df_requests, on='booking_id', how='left')
    df['pt'] = df.apply(lambda row: get_pt(row['pickup_lat'], row['pickup_lon'], row['dropoff_lat'], row['dropoff_lon'], row['action']), axis=1)
    df = df[['booking_id', 'scheduled_arrival_dt', 'action', 'occupancy', 'node_id', 'pt']]
    
    # distance_matrix
    distance_matrix = osrm.request_distance_matrix(df['pt'].tolist())
    
    # calculate number_of_passengers_served
    number_of_passengers_served = len(df[df['action']=='dropoff'])
    
    total_distance, total_distance_without_passengers = 0, 0
    for i in range(len(df)-1):
        total_distance += distance_matrix[i][i+1]
    from_depot = osrm.request_travel_distance({'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON}, df.iloc[0]['pt'])
    to_depot = osrm.request_travel_distance(df.iloc[-1]['pt'], {'lat': config.DEPOT_LAT, 'lon': config.DEPOT_LON})
    total_distance = total_distance + from_depot + to_depot
    total_distance_without_passengers = total_distance_without_passengers + from_depot + to_depot
    
    vehicle_miles_travelled = total_distance / 1609 # convert meters to miles
    vehicle_deadhead_miles = total_distance_without_passengers / 1609
    
    number_of_shared_rides = 0
    passenger_meters_travelled = 0
    for booking_id in df['booking_id'].unique():
        if booking_id != -1:
            pickup = df[(df['booking_id']==booking_id) & (df['action']=='pickup')].iloc[0]
            dropoff = df[(df['booking_id']==booking_id) & (df['action']=='dropoff')].iloc[0]
            dfrn = pd.DataFrame(payload['requests'])
            rn = dfrn[(dfrn['booking_id']==booking_id)].iloc[0]
            num_passengers = rn['am'] + rn['wc']
            passenger_meters_travelled += (distance_matrix[pickup['node_id']][dropoff['node_id']]) * num_passengers
            if (pickup['occupancy'] > 1) or (dropoff['occupancy'] > 0) or (dropoff['node_id'] - pickup['node_id'] != 1):
                number_of_shared_rides += 1
    
    passenger_miles_travelled = passenger_meters_travelled / 1609
    return {
        'VMT': vehicle_miles_travelled,
        'VMT_PMT_Ratio': vehicle_miles_travelled / passenger_miles_travelled,
        'VDM': vehicle_deadhead_miles,
        'number_of_passengers_served': number_of_passengers_served,
        'Shared_Rate': int((number_of_shared_rides / number_of_passengers_served) * 100)
    }

In [8]:
def tsp_insert(i, j, pickup, dropoff, manifest, driver_run_state, time_matrix):
    pickup_location, dropoff_location = deepcopy(pickup), deepcopy(dropoff)
    new_manifest = deepcopy(manifest)
    pickup_location['run_id'], dropoff_location['run_id'] = driver_run_state['run_id'], driver_run_state['run_id']
    new_manifest = new_manifest[0:i] + [pickup_location] + new_manifest[i:j] + [dropoff_location] + new_manifest[j:]
    first_order = driver_run_state['locations_already_serviced']+1
    current_time = driver_run_state['location_dt_seconds']
    current_node_id = driver_run_state['node_id']
    for l in range(len(new_manifest)):
        new_manifest[l]['order'] = l + first_order
        current_time = current_time + time_matrix[current_node_id][new_manifest[l]['node_id']] + config.DWELL_TIME
        if current_time < new_manifest[l]['time_window_start']:
            current_time = new_manifest[l]['time_window_start']
        if (current_time > new_manifest[l]['time_window_end']) or (current_time < driver_run_state['start_time']) or (current_time > driver_run_state['end_time']):
            return {
                'feasible_solution_found': False
            }
        new_manifest[l]['scheduled_time'] = current_time
        current_node_id = new_manifest[l]['node_id']
    return {
        'feasible_solution_found': True,
        'manifest': new_manifest
    }
def tsp_insertion_h(pickup, dropoff, manifest, driver_run_state, time_matrix,payload):
    results = []
    for i in range(len(manifest) + 1):
        for j in range(i, len(manifest) + 1):
            result = tsp_insert(i, j, pickup, dropoff, manifest, driver_run_state, time_matrix)
            if result['feasible_solution_found']:
                results.append(result)
    if results:
        if len(results) == 1:
            return results[0]
        # else get best VMT/PMT ratio
        ratios = []
        for r in results:
            df = pd.DataFrame(r['manifest'])
            df = df.rename(columns={'scheduled_time': 'scheduled_arrival_dt'})
            metrics = manifest_report_h(df,payload)
            ratios.append(metrics['VMT_PMT_Ratio'])
        min_value = min(ratios)
        # Find the index of the minimum value
        min_index = ratios.index(min_value)
        return results[min_index]

    return {
        'feasible_solution_found': False
    }

def online_solver_heuristic(payload):
    '''
    Insertion heurisitc, aiming for the best VMT/PMT ratio.

    :param payload: A dictionary with 'driver_runs', 'pickup', 'dropoff',
                    'time_matrix', and 'date'
    :return: manifest, a dictionary with the optimized route details if a feasible solution
    is found; otherwise, a dictionary indicating no feasible solution.
    '''
    states = list(map(lambda x: {'run_id': x['state']['run_id'], 'total_locations': x['state']['total_locations']}, payload['driver_runs']))
    states = pd.DataFrame(states).sort_values(by='total_locations', ascending=True)
    results = []
    for k, v in states.iterrows():
        driver_run = list(filter(lambda x: x['state']['run_id']==v['run_id'], payload['driver_runs']))[0]
        result = tsp_insertion_h(payload['pickup'], payload['dropoff'], driver_run['manifest'], driver_run['state'], payload['time_matrix'],payload)
        # get all results then return one with best VMT/PMT ratio
        if result['feasible_solution_found']:
            temps = []
            for doc in result['manifest']:
                temp = {key: doc[key] for key in ['action', 'booking_id', 'node_id', 'run_id', 'scheduled_time']}
                temps.append(temp)
            result['manifest'] = temps
            result['run_id'] = v['run_id']
            results.append(result)
    if results:
        if len(results) == 1:
            return results[0]
        ratios = []
        for r in results:
            df = pd.DataFrame(r['manifest'])
            df = df.rename(columns={'scheduled_time': 'scheduled_arrival_dt'})
            metrics = manifest_report_h(df,payload)
            ratios.append(metrics['VMT_PMT_Ratio'])

        min_value = min(ratios)
        # Find the index of the minimum value
        min_index = ratios.index(min_value)
        return results[min_index]
    
    result = {
        'feasible_solution_found': False
    }
    return result

In [13]:

solver_result = online_solver_heuristic(payload)
if solver_result['feasible_solution_found']:
    print(solver_result)
else:
    print("Feasible Solution Not Found")


{'feasible_solution_found': True, 'manifest': [{'action': 'pickup', 'booking_id': 1129856, 'node_id': 355, 'run_id': 9, 'scheduled_time': 27000}, {'action': 'dropoff', 'booking_id': 1129856, 'node_id': 356, 'run_id': 9, 'scheduled_time': 27938}], 'run_id': 9}
